# Table of Contents


* [Importing libraries](#importing-libraries)
* [Loading Data](#loading-data)



<a id="importing-libraries"></a>
## Importing Libraries


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from os import listdir
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Finding out all the files that we can analyse and list them for loading and further exploration. 
arr = os.listdir('/kaggle/input/healthcare-analytics/Train')
print(arr)

<a id="loading-data"></a>
## Loading Data

In [ ]:


#Loading all the files for further explorations.
train_df = pd.read_csv('/kaggle/input/healthcare-analytics/Train/Train.csv')
first_health_df = pd.read_csv('/kaggle/input/healthcare-analytics/Train/First_Health_Camp_Attended.csv')
second_health_df = pd.read_csv('/kaggle/input/healthcare-analytics/Train/Second_Health_Camp_Attended.csv')
third_health_df = pd.read_csv('/kaggle/input/healthcare-analytics/Train/Third_Health_Camp_Attended.csv')
health_camp_detail_df = pd.read_csv('/kaggle/input/healthcare-analytics/Train/Health_Camp_Detail.csv')
patient_profile_df = pd.read_csv('/kaggle/input/healthcare-analytics/Train/Patient_Profile.csv')
test_init = pd.read_csv('/kaggle/input/healthcare-analytics/Train/test.csv')


### Analyzing the First Health Camp data

In [ ]:
first_health_df.head()

### Analyzing the Second Health Camp data

In [ ]:
second_health_df.sort_values(['Health Score'], ascending = True)

### Analyzing the Third Health Camp data 

In [ ]:
third_health_df.head()

In [ ]:
third_health_df.shape

In [ ]:
third_health_df.sort_values(['Number_of_stall_visited'], ascending = True)

In [ ]:
positive_outcome_third = third_health_df[third_health_df['Number_of_stall_visited'] > 0]

All the positive outcomes from the third health camp are stored in the positive_outcome_third table below - we know that a positive outcome constitutes a number of stalls visited bigger than 11 - so thats how we create this table:

In [ ]:
positive_outcome_third

### Finding the pairs of Patient_ID and Health_camp_ID with positive outcomes that figure out in camps 1, 2 or 3 and then creating a data frame where we store them

In [ ]:
positive_1 = first_health_df.drop(["Donation", 'Health_Score', 'Unnamed: 4'], axis=1)


In [ ]:
positive_2 = second_health_df.drop(['Health Score'], axis=1)


In [ ]:
positive_3 = positive_outcome_third.drop(['Number_of_stall_visited', 'Last_Stall_Visited_Number'], axis = 1)

Concatenating first 2 columns of each of the above 3 databases into one positive outcome:

In [ ]:
positive_outcomes = pd.concat([positive_1, positive_2, positive_3], axis = 0)

We observe that a patient can attend more than one data camp, or none. The challenge is to observe for each camp, each of the patients whether in the train or test data set has a positive or negative outcome. For that we will have find a way to asses if for each pair in the train set if the outcome was positive or not. Once we have the table that contains all the positive outcomes observed - we can use it to find out if each pair (patient_ID, Health_Camp_ID) in the train set has a positive or negative outlook. 

In [ ]:
#create tuple column with (Patient_ID, Health_Camp_ID) - we see we have 20534 positive outcomes.
positive_outcomes['patient_camp'] = list(zip(positive_outcomes.Patient_ID, positive_outcomes.Health_Camp_ID))

In [ ]:
positive_outcomes.head()

In [ ]:
train_df['patient_camp_train'] = list(zip(train_df.Patient_ID, train_df.Health_Camp_ID))

In [ ]:
train_df.head()

In [ ]:
positive_list = list(positive_outcomes.patient_camp)

In [ ]:
len(positive_list)

### Details of the Healts Camps

In [ ]:
health_camp_detail_df.info

### Patient Profiles

In [ ]:
patient_profile_df.head()

In [ ]:
patient_profile_df.info()

We aim to create function that will provide us with a list of camps with positive outcomes atendeded by each patient - and we have the data already stored in the positive_outcomes table:

In [ ]:
positive_outcomes.head()


In [ ]:
# we have 20534 patients and camps touples that qualify as - positive outcomes:
len(positive_list)

In [ ]:
positive_list[:3]

In [ ]:
#accessing only the camp portion of a tuple:
positive_list[0][1]

In [ ]:
# how many unique patients we have that have positive outcomes?
unique_patient_ID_with_positive = list(positive_outcomes.Patient_ID.unique())

In [ ]:
unique_patient_ID_with_positive[:3]

In [ ]:
#we have only 11069 patients with positive outcomes:
len(unique_patient_ID_with_positive)

In [ ]:
#Creating a function that shows all camps with positive outcome attended by a patient:

In [ ]:
def camps(patient):
    camps = []
    i = 0
    while i < 20533:
        if positive_list[i][0] == patient:
            camps.append(positive_list[i][1])
        i += 1
    return camps


Below started to create a dictionary of camps for each patient - this still does not work - figure out why - I assume for all the null results for the camp function - need to define that better?


In [ ]:
#patient_camps_attended = { 'patient_id': patient_profile_df.Patient_ID,
#                          'camps': camps(patient_profile_df.Patient_ID)}

In [ ]:
camps(0)

In [ ]:
train_df.size

Will create a function that takes 2 parameters - camp and patient - so it  tests if a patient and camp couple is succesfull. If the camp function of the patient is null - we return 0 as it is unsuccesful. But - if the patient did indeed attend a succesful camp - we test to see if this specific camp in the function is in the patients list of camps: 


In [ ]:

def outcome(patient, camp):
    if len(camps(patient)) == 0:
        return 0
    
    elif camp in camps(patient):
        return 1 
    else:
        return 0
    

In [ ]:
outcome(485679, 6555)

In [ ]:
outcome(485679, 6578)

In [ ]:
train_df.head()

We create a list where we check the outcome for each Patient_ID, Health_Camp_ID in the train dataframe:

In [ ]:
outcomes = []
for patient_camp_tuple in train_df.patient_camp_train:
    outcomes.append(outcome(patient_camp_tuple[0], patient_camp_tuple[1])) 
outcomes

In [ ]:
df = pd.DataFrame(outcomes,columns=['Outcome'])

In [ ]:
df.head()

In [ ]:
type(df.Outcome)

In [ ]:
type(train_df.Var1)

In [ ]:
new = pd.concat((train_df, df), axis = 1)

In [ ]:
new.head()

In [ ]:
new.Outcome.unique()

In [ ]:
new.Outcome[10014]

In [ ]:
new.Outcome.sum()

### Analyzing the Train and Test data

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_init.shape

In [ ]:
test_init.head()

In [ ]:
new.head()

In [ ]:
new.head()

In [ ]:
### Preping and Spliting the data 

In [ ]:
# Separate features and labels
X = new.drop(["Patient_ID", "Health_Camp_ID","Registration_Date", "Outcome", "patient_camp_train"], axis = 1)
y = df
#y = new.drop(["Patient_ID", "Health_Camp_ID","Registration_Date","patient_camp_train", "Var1", "Var2", "Var3", "Var4", "Var5"], axis = 1)

In [ ]:
X.head()

In [ ]:
X.describe()

Analyzing the variables used as features for the model:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

features = ['Var1','Var2','Var3','Var4','Var5']
for col in features:
    new.boxplot(column=col, by='Outcome', figsize=(6,6))
    plt.title(col)
plt.show()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training cases: %d\nTest cases: %d' % (X_train.size, X_test.size))

In [ ]:
# Train the model
from sklearn.linear_model import LogisticRegression

# Set regularization rate
reg = 0.01

# train a logistic regression model on the training set
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train.values.ravel())
                                                            
                                                        
print (model)

In [ ]:
predictions = model.predict(X_test)
print('Predicted labels: ', predictions)
print('Actual labels:    ' ,y_test)

In [ ]:
predictions.sum()

In [ ]:
y_test.Outcome.sum()

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_test, predictions))

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Overall Precision:",precision_score(y_test, predictions))
print("Overall Recall:",recall_score(y_test, predictions))

ROC AUC score, confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Print the confusion matrix
cm = confusion_matrix(y_test, predictions)
print (cm)

In [ ]:
y_scores = model.predict_proba(X_test)
print(y_scores)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

The model above has a very poor ROC score - barely better than 0.5. :et try to normalize the data and see if that will improve a bit the outcome:

In [ ]:
# Train the model
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define preprocessing for numeric columns (normalize them so they're on the same scale)
numeric_features = [0,1,2,3,4]
numeric_transformer = Pipeline(steps=[
    ('scaler', PowerTransformer())])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', LogisticRegression(C=1/reg, solver="liblinear"))])


# fit the pipeline to train a logistic regression model on the training set
model_pipe = pipeline.fit(X_train, (y_train.values.ravel()))
print (model_pipe)

In [ ]:
# Get predictions from test data
predictions_pipe = model_pipe.predict(X_test)

# Get evaluation metrics
cm = confusion_matrix(y_test, predictions_pipe)
print ('Confusion Matrix:\n',cm, '\n')
print('Accuracy:', accuracy_score(y_test, predictions_pipe))
print("Overall Precision:",precision_score(y_test, predictions_pipe))
print("Overall Recall:",recall_score(y_test, predictions_pipe))
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

# calculate ROC curve
y_scores = model_pipe.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()


Didnt improve at all - trying Random Forest:


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', RandomForestClassifier(n_estimators=100))])

# fit the pipeline to train a random forest model on the training set
model = pipeline.fit(X_train, (y_train.values.ravel()))
print (model)

In [ ]:
predictions = model.predict(X_test)
cm = confusion_matrix(y_test, predictions)
print ('Confusion Matrix:\n',cm, '\n')
print('Accuracy:', accuracy_score(y_test, predictions))
print("Overall Precision:",precision_score(y_test, predictions))
print("Overall Recall:",recall_score(y_test, predictions))
auc = roc_auc_score(y_test,y_scores[:,1])
print('\nAUC: ' + str(auc))

# calculate ROC curve
y_scores = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

Lets try and see how our model performs on the test data - although with this AUC I wont be submitting anything.

In [ ]:
test_init.head()

In [ ]:
X_test_test = test_init.drop(["Patient_ID", 'Health_Camp_ID', "Registration_Date"], axis = 1)

In [ ]:
X_test_test.head()

Create a column for tuples of the Patient_ID, Health_Camp_ID:

In [ ]:
test_init['patient_camp_train'] = list(zip(test_init.Patient_ID, test_init.Health_Camp_ID))

In [ ]:
predictions_test = model.predict(X_test_test)

In [ ]:
predictions_test

In [ ]:
outcomes_test = pd.DataFrame(predictions_test, columns = ["Outcome"])

In [ ]:
outcomes_test

In [ ]:
test_for_output = test_init.drop(["Registration_Date", "Var1", "Var2", "Var3", "Var4", "Var5", "patient_camp_train"], axis = 1)
test_for_output

In [ ]:
output_df = pd.concat([test_for_output, outcomes_test], axis = 1)

In [ ]:
output_df 

In [ ]:
output_df.to_csv("Output.csv", index = False)

In [ ]:
testing_output_csv = pd.read_csv('/kaggle/working/Output.csv')


In [ ]:
testing_output_csv